In [30]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [31]:
data1 = pd.read_csv('data/raw_data/ms_data_raw.csv').iloc[:,1:]
data2 = pd.read_csv('data/raw_data/2023년도_학교기본정보(중)_전체.csv')

In [32]:
to_replace = {
    '상일여자중학교':'상일중학교',
    '미양중학교':'솔샘중학교',
    '보영여자중학교':'한빛누리중학교',
    '의정부서중학교':'다온중학교',
    '송탄제일중학교':'라온중학교',
    '협성중학교':'협성경복중학교',
    '재능중학교':'인천재능중학교',
    '울산여자중학교':'가온중학교',
    '조치원여자중학교':'세종중학교',
    '북평여자중학교':'예람중학교',
    '묵호여자중학교':'하랑중학교',
    '삼척여자중학교':'청아중학교',
    '설악여자중학교':'설온중학교',
    '속초여자중학교':'속초해랑중학교',
    '양구여자중학교':'석천중학교',
    '북원여자중학교':'북원중학교',
    '춘천여자중학교':'봄내중학교',
    '황지여자중학교':'세연중학교',
    '금암중학교':'계룡중학교',
    '건양중학교':'건양대학교병설건양중학교',
    '계광중학교':'천안계광중학교',
    '원이중학교이원분교장':'원이중학교이원분교',
    '고창여자중학교':'자유중학교',
    '금성여자중학교':'금성중학교',
    '김제여자중학교':'한들중학교',
    '부안여자중학교':'낭주중학교',
    '전주효문여자중학교':'전주효문중학교',
    '정읍여자중학교':'샘고을중학교',
    '정일여자중학교':'정일중학교',
    '장흥여자중학교':'장흥향원중학교',
    '자인여자중학교':'경산제일중학교',
    '장목중학교':'장목예술중학교',
    '경상대학교사범대학부설중학교':'경상국립대학교사범대학부설중학교',
    '삼산승영중학교':'승영중학교',
    '대전신일여자중학교':'대전신일중학교'
}

In [33]:
data1['학교명'] = data1['학교명'].replace(to_replace)

In [34]:
data1['학교명'] = data1['도']+' '+data1['학교명']

In [35]:
idx = data1[data1['학교명'].isin(data1['학교명'].value_counts()[data1['학교명'].value_counts()>1].index)].index
data1.loc[idx, '학교명'] = data1.loc[idx, '학교명'] + ' ' + data1.loc[idx, '시군구']

In [36]:
print(data1['학교명'].nunique(), data1.shape[0])

3004 3004


In [37]:
data2 = data2[~data2['학교명'].str.contains('예술|국제|체육')]

In [38]:
data2['지역'] = data2['지역'].fillna(data2['시도교육청'])

In [39]:
data2['지역'] = data2['지역'].apply(lambda x: x.split()[0])
for k,v in {'특별':'','자치':'','광역':'','도':'','시':'','경상':'경','전라':'전','충청':'충','교육청':''}.items():
    data2['지역'] = data2['지역'].str.replace(k,v)

In [40]:
data2['학교명'] = data2['지역'] + ' ' + data2['학교명']

In [41]:
idx = data2[data2['학교명'].isin(data2['학교명'].value_counts()[data2['학교명'].value_counts()>1].index)].index
data2.loc[idx, '학교명'] = data2.loc[idx, '학교명'] + ' ' + data2.loc[idx, '학교도로명 주소'].apply(lambda x: x.split()[1])

In [42]:
data2 = data2.merge(data1, on='학교명')[['학교명','위도','경도','평균','졸업자수']]

In [43]:
data2['평균'] = data2['평균'].str.replace('%','').astype(float)
data2['졸업자수'] = data2['졸업자수'].str.replace('명','').astype(int)

In [44]:
data2[data2['위도']<33]

,학교명,위도,경도,평균,졸업자수
417,부산 학산여자중학교,12.909518,129.095183,76.9,91


In [45]:
data2.loc[417, ['위도','경도']] = [35.2047506, 129.0962459]

In [49]:
data2[data2['경도']<124]

,학교명,위도,경도,평균,졸업자수
2758,경남 곤양중학교,35.0335,112.758238,61.3,18


In [50]:
data2.loc[2758, ['위도','경도']] = [35.0598195, 127.973135]

In [53]:
data2.isna().sum()

학교명     0
위도      0
경도      0
평균      0
졸업자수    0
dtype: int64

In [52]:
data2.to_csv('data/ms_data.csv', encoding='cp949', index=False)